In [ ]:
#dans le terminal sur vs code, copier coller la ligne suivante
#pip install selenium

In [ ]:
# -*- coding: utf-8 -*-
import csv, time, random, os
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# =================== CONFIGURATION ===================
CSV_FILE = "carrefour_daily_monitoring.csv"
SEARCH_URL = "https://www.carrefour.fr/s?q="
PRODUCT_REFS = [
    '0195949823763','0195949822865','0195949821967',
    '0195949724169','0195949723216','0195949722264','0195949773488',
    '0195949807336','0195949037863','0195949036965','0195949036064',
    '0195949042539','0195949041631','0195949040733','0195949020735',
    '0195949049699','0194253408253','0194253409809','0194253411550',
    '0194253373476','0194253374626','0194253375777','0194253401179',
    '0194253403814','0194253405139','0194253379942','0194253381686',
    '8806095851792','8806095851488','8806095857725','8806095857602',
    '8806095860107','8806095860237','8806095859934','8806095851433',
    '8806095859910','8806095859743','8806095859880','8806095299785',
    '8806095306797','8806095307633','8806095414485','8806095414478'
]

def build_edge_driver_invisible():
    options = Options()
    options.add_argument("--headless=new") 
    options.add_argument("--start-maximized")
    options.add_argument("--window-size=1920,1080")
    options.add_experimental_option('excludeSwitches', ['enable-logging', 'enable-automation'])
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0")
    
    driver = webdriver.Edge(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

def job():
    print(f"\n--- 🕒 Lancement du cycle : {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} ---")
    driver = build_edge_driver_invisible()
    results = []
    
    try:
        driver.get("https://www.carrefour.fr/")
        time.sleep(5)
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()
        except: pass

        for ref in PRODUCT_REFS:
            try:
                driver.get(f"{SEARCH_URL}{ref}")
                time.sleep(random.uniform(5, 8))
                
                wait = WebDriverWait(driver, 15)
                product_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[href*='{ref}']")))
                product_url = product_link.get_attribute("href")
                driver.get(product_url)
                
                wait.until(EC.presence_of_element_located((By.ID, "product-title-desktop")))
                
                name = driver.find_element(By.ID, "product-title-desktop").text.strip()
                price_raw = driver.find_element(By.CSS_SELECTOR, ".product-price__amount").text
                price = price_raw.replace('\n', '').replace(' ', '').replace(',', '.')

                print(f"✅ Extrait : {name} - {price}")
                
                results.append(["Carrefour", name, "Carrefour", "Standard", price, "N/A", datetime.now().strftime("%Y-%m-%d %H:%M:%S"), product_url])
            except Exception:
                print(f"⚠️ Échec pour l'EAN {ref}")
        
        if results:
            file_exists = os.path.isfile(CSV_FILE)
            with open(CSV_FILE, "a", newline="", encoding="utf-8") as f:
                w = csv.writer(f, delimiter=";")
                if not file_exists:
                    w.writerow(["Platform", "Product Name", "Seller", "Delivery Info", "Price", "Seller Rating", "Timestamp", "Source URL"])
                w.writerows(results)
            print(f"📊 Données sauvegardées dans {CSV_FILE}")

    finally:
        driver.quit()

# =================== BOUCLE INFINIE ===================
if __name__ == "__main__":
    while True:
        job()
        
        prochain_passage = datetime.now() + timedelta(hours=24)
        print(f"💤 Cycle terminé. Mise en veille pour 24h.")
        print(f"📅 Prochaine exécution prévue le : {prochain_passage.strftime('%d/%m/%Y à %H:%M:%S')}")
        
        # Pause de 86400 secondes (24 heures)
        time.sleep(86400)